<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(Urclass Quiz) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 고르세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [1]:
!pip install pandas_profiling

     |████████████████████████████████| 259 kB 23.8 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 3.2.0
    Uninstalling pandas-profiling-3.2.0:
      Successfully uninstalled pandas-profiling-3.2.0


In [4]:
! pip install kaggle

In [5]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c prediction-of-h1n1-vaccination

  0% 0.00/1.59M [00:00<?, ?B/s]
100% 1.59M/1.59M [00:00<00:00, 37.8MB/s]


In [9]:
! unzip prediction-of-h1n1-vaccination

Archive:  prediction-of-h1n1-vaccination.zip
  inflating: submission.csv          
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_labels.csv        


In [10]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42154 entries, 0 to 42153
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 33734 non-null  float64
 1   h1n1_knowledge               33734 non-null  float64
 2   behavioral_antiviral_meds    33635 non-null  float64
 3   behavioral_avoidance         33482 non-null  float64
 4   behavioral_face_mask         33710 non-null  float64
 5   behavioral_wash_hands        33683 non-null  float64
 6   behavioral_large_gatherings  33640 non-null  float64
 7   behavioral_outside_home      33633 non-null  float64
 8   behavioral_touch_face        33571 non-null  float64
 9   doctor_recc_h1n1             40269 non-null  float64
 10  doctor_recc_seasonal         40269 non-null  float64
 11  chronic_med_condition        40837 non-null  float64
 12  child_under_6_months         32705 non-null  float64
 13  health_insurance

In [4]:
target = 'vacc_h1n1_f'

train = pd.merge(pd.read_csv('train.csv'), 
                 pd.read_csv('train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('submission.csv')

from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)

def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
#     selected_cols = df.select_dtypes(include=['number', 'object'])
#     colnames = selected_cols.columns.tolist()
#     labels = selected_cols.nunique()
    
#     selected_features = labels[labels <= 30].index.tolist()
#     df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df

train = engineer(train)
val = engineer(val)
test = engineer(test)


features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

!pip install category_encoders

from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

# ordinal encoding
enc = OrdinalEncoder(handle_missing="value")
enc.fit(X_train, y_train)
enc.category_mapping

     |████████████████████████████████| 86 kB 4.6 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[{'col': 'opinion_h1n1_vacc_effective',
  'data_type': dtype('O'),
  'mapping': Somewhat Effective      1
  Not Very Effective      2
  Very Effective          3
  Not At All Effective    4
  Dont Know               5
  NaN                     6
  Refused                 7
  dtype: int64},
 {'col': 'opinion_h1n1_risk',
  'data_type': dtype('O'),
  'mapping': Somewhat Low     1
  Somewhat High    2
  Very Low         3
  NaN              4
  Very High        5
  Dont Know        6
  Refused          7
  dtype: int64},
 {'col': 'opinion_h1n1_sick_from_vacc',
  'data_type': dtype('O'),
  'mapping': Not At All Worried    1
  Not Very Worried      2
  Dont Know             3
  Somewhat Worried      4
  Very Worried          5
  NaN                   6
  Refused               7
  dtype: int64},
 {'col': 'agegrp', 'data_type': dtype('O'), 'mapping': 18 - 34 Years         1
  35 - 44 Years         2
  65+ Years             3
  55 - 64 Years         4
  45 - 54 Years         5
  6 Months - 9 Ye

### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **(Urclass Quiz) 캐글 Leaderboard의 본인 Score를 제출하세요.**

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

검증 정확도:  0.8254062388803226


## 🔥 도전과제(Github - Discussion)

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 2개 이상 사용해 결과를 공유해 보시고, 다음 질문에 대해 서로 논의해 보세요. 

- 사용하신 encoder는 각각 어떤 장단점을 갖고 있으며, 어떤 상황에서 사용하면 좋을까요? 

### 4) 왜 트리모델에서는 ordinal encoding을 주로 사용하며 (one-hot encoding대신), 범주형 자료를 ordinal encoding으로 사용해도 되는 이유는 무엇이라고 생각하시는지 논의해 보세요


## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)